In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Get distance on battery

In [11]:
header, configs = load_header_config('aglog_header.csv')
df = pd.read_csv('vin_LMGGN1S51F1000990.csv', header=None, names=header, index_col=False)
df.iloc[:, 1] = pd.to_datetime(df.iloc[:, 1], unit='ms', utc=True).dt.tz_convert('Asia/Hong_Kong')
df = df.sort_values(by=df.columns[1])
print(df.shape)
# df = df.drop_duplicates(subset=df.columns[1])
df.loc[:, 'delta'] = df.iloc[:, 1].diff().fillna(timedelta(seconds=10))
df.loc[:, 'delta'] = df['delta'].apply(lambda x: x.seconds)
# df.loc[:, 'NAcount'] = df.isnull().sum(axis=1)
df = df[df['delta'] > 5]
df = df.reset_index(drop=True)
print(df.shape)
df1 = df

(26478, 85)
(26374, 86)


In [19]:
df = df1.iloc[:, [1, 11, 16, 57, 60]].copy()
df.columns = ['TDATE', 'BMS_BATTSOC', 'CCS_CHARGECUR', 'HCU_AVGFUELCONSUMP', 'ICM_TOTALODOMETER']
filter1 = df['ICM_TOTALODOMETER'] > 0 
filter2 = (df['CCS_CHARGECUR'] < 0.1) | (df['CCS_CHARGECUR'].isna())
# filter3 = (df['HCU_BATCHRGDSP'] == 0) | (df['HCU_BATCHRGDSP'].isna())
filter_driving = filter1 & filter2

filter4 = df['HCU_AVGFUELCONSUMP'] < 0.000001
filter5 = df['BMS_BATTSOC'] > 20

filter_drive_batt = filter4 & filter5
df['flag'] = (filter_driving & filter_drive_batt).astype(int)
# flag = 1 means driving on battery
df = df[df['flag'] == 1].reset_index()
df.loc[:, 'index'] = df['index'].diff().fillna(2.0)
indices = df.index[df['index'] != 1.0].tolist()
if df['index'].iloc[-1] == 1:
    print(indices[-3:])
    indices.append(df.index[-1])
    print(indices[-3:])

[3012, 3035, 3063]
[3035, 3063, 3070]


## Load string data as timestamp object

In [63]:
# Wrong!
print(pd.to_datetime('2015-02-01 00:00:00+0800').tz_localize('Asia/Hong_Kong'))

# when str has timezone info, use utc=True and convert to local timezone
print(pd.to_datetime('2015-02-01 00:00:00+0800', utc=True).tz_convert('Asia/Hong_Kong'))

# when str has no timezone info, if it is already in local time, just localize it with correct timezone
print(pd.to_datetime('2015-02-01 00:00:00').tz_localize('Asia/Hong_Kong'))

2015-01-31 16:00:00+08:00
2015-02-01 00:00:00+08:00
2015-02-01 00:00:00+08:00


# test for daily run

In [2]:
COL_DEPENDENCY_DICT = {
    'distance_driven': ['TDATE', 'ICM_TOTALODOMETER'],
    'distance_on_battery': ['TDATE', 'ICM_TOTALODOMETER', 'BMS_BATTSOC', 'HCU_AVGFUELCONSUMP',
                            'HCU_BATCHRGDSP', 'CCS_CHARGECUR'],
    'distance_on_fuel': ['TDATE', 'ICM_TOTALODOMETER', 'BMS_BATTSOC', 'HCU_AVGFUELCONSUMP',
                         'HCU_BATCHRGDSP', 'CCS_CHARGECUR'],
    'odometer_offset': ['TDATE', 'ICM_TOTALODOMETER'],
    'speed_average': ['TDATE', 'BCS_VEHSPD'],
    'speed_max': ['TDATE', 'BCS_VEHSPD'],
    'soc_range_charge_start': ['TDATE', 'CCS_CHARGECUR', 'BMS_BATTSOC'],
    'soc_range_charge_end': ['TDATE', 'CCS_CHARGECUR', 'BMS_BATTSOC'],
    'charged_times': ['TDATE', 'CCS_CHARGECUR', 'BMS_BATTSOC'],
    'charged_energy': ['TDATE', 'CCS_CHARGEVOLT', 'CCS_CHARGECUR'],
    'charging_hours': ['TDATE', 'CCS_CHARGECUR'],
    'driving_hours': ['TDATE', 'ICM_TOTALODOMETER', 'CCS_CHARGECUR', 'HCU_BATCHRGDSP'],
    'energy_throughput': ['TDATE', 'BMS_BATTCURR', 'BMS_BATTVOLT'],
    'battery_capacity': ['TDATE', 'BMS_BATTCURR'],
    'battery_temp_avg': ['BMS_BATTTEMPAVG'],
    'battery_temp_max': ['BMS_BATTTEMPMAX'],
    'battery_temp_min': ['BMS_BATTTEMPMIN'],
    'cum_fuel_consumption': ['TDATE', 'ICM_TOTALODOMETER', 'BMS_BATTSOC', 'HCU_AVGFUELCONSUMP',
                             'HCU_BATCHRGDSP', 'CCS_CHARGECUR'],
    'cum_elec_energy_consumption': ['TDATE', 'BMS_BATTCURR', 'BMS_BATTVOLT'],
    'elec_energy_efficiency': ['TDATE', 'ICM_TOTALODOMETER', 'BMS_BATTSOC', 'HCU_AVGFUELCONSUMP',
                               'HCU_BATCHRGDSP', 'CCS_CHARGECUR', 'BMS_BATTCURR', 'BMS_BATTVOLT'],
    'numStat_fuel_efficiency': ['HCU_AVGFUELCONSUMP'],
    'numStat_speed_diff': ['SPD_DIF'],
    'numStat_cell_volt_diff': ['BMS_CELLVOLTDIFF'],
    'numStat_SOC': ['BMS_BATTSOC'],
    'numStat_acce_pedal': ['EMS_ACCPEDALPST'],
    'numStat_battery_voltage': ['BMS_BATTVOLT'],
    'catStat_brake_pedal': ['EMS_BRAKEPEDALST']
}
col_dict = {1: 'TDATE', 2: 'SDATE', 4: 'BMS_BATTCURR', 5: 'BMS_BATTVOLT', 8: 'BMS_CELLVOLTMAX',
            9: 'BMS_CELLVOLTMIN', 11: 'BMS_BATTSOC', 12: 'BMS_BATTTEMPAVG', 13: 'BMS_BATTTEMPMAX',
            14: 'BMS_BATTTEMPMIN', 15: 'CCS_CHARGEVOLT', 16: 'CCS_CHARGECUR', 52: 'EMS_ACCPEDALPST',
            53: 'EMS_BRAKEPEDALST', 57: 'HCU_AVGFUELCONSUMP', 58: 'HCU_BATCHRGDSP', 59: 'BCS_VEHSPD', 
            60: 'ICM_TOTALODOMETER'}

In [27]:
from vehicle import *
from veh_stats import *
import findspark
findspark.init("/usr/hdp/current/spark2-client")
import pyspark
from pyspark.sql import SQLContext, HiveContext
conf = pyspark.SparkConf().setAll([('spark.app.name', 'daily_stats_run'), # App Name
                                   ('spark.master', 'yarn'),              # spark run mode: locally or remotely
                                   ('spark.submit.deployMode', 'client'), # deploy in yarn-client or yarn-cluster
                                   ('spark.executor.memory', '10g'),      # memory allocated for each executor
                                   ('spark.memory.fraction', '0.8'),      # fraction of memory for execution and storage
                                   ('spark.executor.cores', '3'),         # number of cores for each executor
                                   ('spark.executor.instances', '40'),    # number of executors in total
                                   ('spark.yarn.am.memory','20g')])       # memory for spark driver

In [4]:
def transform_to_tuple(line):
    """
    output is a tuple, key is vin, value is all columns in col_dict
    """
    fields = line.split(",")
    vin = fields[0]
    otherfields = {}
    for col_index, col in col_dict.items():
        this_value = fields[int(col_index)]
        otherfields[col] = this_value
    return vin, otherfields

def compute_stats(x, freq):
    """
    x is a list of tuple, (vin, other_fields)
    return is (vin, stats). stats will be a dict {'vin1': {'daily': stats},
                                                  'vin1': 'hourly': stats}...
    """
    df = pd.DataFrame(list(x[1]))
    df['VIN'] = x[0]
    stats = sorted(list(COL_DEPENDENCY_DICT.keys()))
    freqs = freq
    veh = Vehicle(df)
    veh.add_cellvoltdiff()
    veh.add_speed_diff()
    res = get_freq_based_stats(veh.df, veh.vin, stats, freqs)
    return veh.vin, res

In [5]:
def get_daily_stats(data_file, freq):
    rdd = sc.textFile(data_file).filter(lambda line: len(line.split(',')) in [85, 86])
    res = rdd.map(transform_to_tuple).groupByKey().map(lambda x: compute_stats(x, freq)).collect() 
    vals = OrderedDict(res).values()
    df = pd.DataFrame()
    for i in range(len(vals)):
        df = pd.concat([df, pd.DataFrame(vals[i][freq])])
    print(data_file + ' done.')
    return df

In [29]:
sc.stop()
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sc.addPyFile('/home/stang/user-profile/stats-spark/veh.zip')
sc

<SparkContext master=yarn appName=daily_stats_run>

In [11]:
sc.getConf().getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.driver.port', u'46792'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.yarn.am.memory', u'20g'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.driver.appUIAddress', u'http://172.15.7.170:4040'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.ui.proxyBase', u'/proxy/application_1517606397794_1022'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  u'http://namenode:8088/proxy/application_1517606397794_1024'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.history.FsHistoryProvider'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.executor.cores', u'3'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.yarn.hist

In [30]:
conf1 = pyspark.SparkConf().setAll([('spark.app.name', 'export_to_hive'), 
                                    ('spark.executor.instances', '8')])

In [31]:
# sc.stop()
sc = pyspark.SparkContext(conf=conf1)
sc.getConf().getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.yarn.am.memory', u'20g'),
 (u'spark.ui.proxyBase', u'/proxy/application_1517606397794_1025'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  u'http://namenode:8088/proxy/application_1517606397794_1026'),
 (u'spark.driver.appUIAddress', u'http://172.15.7.170:4040'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.history.FsHistoryProvider'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.executor.cores', u'3'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.yarn.historyServer.address', u'namenode:1808

In [54]:
date = 20150118
daily_csv_file = 'hdfs://namenode:8020/data/ag/by-day/ag_{}.csv'.format(date)

In [55]:
%%time
res_daily = get_daily_stats(daily_csv_file, 'daily')

hdfs://namenode:8020/data/ag/by-day/ag_20150118.csv done.
CPU times: user 110 ms, sys: 7.91 ms, total: 117 ms
Wall time: 6.2 s


In [80]:
res_daily

,SOC_max,SOC_mean,SOC_min,SOC_std,acce_pedal_count,acce_pedal_max,acce_pedal_mean,acce_pedal_min,acce_pedal_std,battery_capacity,battery_temp_avg,battery_temp_max,battery_temp_min,battery_voltage_count,battery_voltage_max,battery_voltage_mean,battery_voltage_min,battery_voltage_std,brake_pedal_count,brake_pedal_freq,brake_pedal_top,brake_pedal_unique,cell_volt_diff_count,cell_volt_diff_max,cell_volt_diff_mean,cell_volt_diff_min,cell_volt_diff_std,charged_energy,charged_times,charging_hours,cum_elec_energy_consumption,cum_fuel_consumption,distance_driven,distance_on_battery,distance_on_fuel,driving_hours,elec_energy_efficiency,energy_throughput,fuel_efficiency_count,fuel_efficiency_max,fuel_efficiency_mean,fuel_efficiency_min,fuel_efficiency_std,odometer_offset,soc_range_charge_end_count,soc_range_charge_end_max,soc_range_charge_end_mean,soc_range_charge_end_min,soc_range_charge_end_std,soc_range_charge_start_count,soc_range_charge_start_max,soc_range_charge_start_mean,soc_range_charge_start_min,soc_range_charge_start_std,speed_average,speed_count,speed_diff_count,speed_diff_max,speed_diff_mean,speed_diff_min,speed_diff_std,speed_max,vin
0,99.7,95.818421,93.8,1.689553,68.0,27.832,3.170588,0.0,7.489834,1.258333,20.000000,21.0,20.0,76.0,359.0,334.763158,0.0,68.395101,68.0,45.0,1.0,2.0,76.0,0.09,0.021842,0.00,0.022194,NaN,0,NaN,0.434765,8.800000,1.0,0,1.0,0.191667,NaN,0.434765,76.0,8.8,8.800000,8.8,3.576320e-15,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,9.645996,64,65.0,27.73125,-1.093143e-16,-26.38125,8.817053,46.63125,LMGGN1S53E1000035
0,100.0,56.525636,30.1,16.897383,772.0,72.128,4.193181,0.0,9.042312,43.611111,20.796639,24.0,16.0,2321.0,362.0,342.806118,0.0,27.155529,772.0,522.0,0.0,2.0,2321.0,0.22,0.012206,0.00,0.022681,8.331073,1,4.230556,7.049156,192.150711,33.0,0,33.0,2.147222,NaN,14.867685,2321.0,6.0,5.822749,1.9,7.265646e-01,0.0,1.0,100.0,100.0,100.0,NaN,1.0,30.2,30.2,30.2,NaN,16.801398,684,763.0,46.18125,-2.064220e-03,-48.71250,9.991824,77.11875,LMGGN1S58E1000032
0,100.0,48.614440,20.7,27.035398,1105.0,80.752,3.783421,0.0,9.217188,57.912500,16.519601,21.0,12.0,3061.0,368.0,343.184907,0.0,23.368974,1105.0,775.0,0.0,2.0,3061.0,0.12,0.011107,0.00,0.012802,10.539841,1,5.366667,4.034926,199.826952,31.0,0,31.0,3.069444,NaN,19.891951,3061.0,6.6,6.446031,6.0,2.395968e-01,0.0,1.0,100.0,100.0,100.0,NaN,1.0,22.8,22.8,22.8,NaN,13.760789,811,1096.0,27.33750,-1.180429e-03,-27.39375,5.878786,63.95625,LMGGN1S56E1000045
0,31.0,26.583596,23.0,2.224443,293.0,43.120,4.501980,0.0,8.244877,10.552778,16.679811,19.0,14.0,317.0,349.0,324.880126,0.0,67.386976,293.0,175.0,0.0,2.0,317.0,0.05,0.012618,0.00,0.008215,NaN,0,NaN,2.210471,85.376972,15.0,0,15.0,0.816667,NaN,3.575539,317.0,6.2,5.691798,4.8,5.657377e-01,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,21.392908,245,282.0,29.47500,-3.171543e-02,-30.93750,7.480024,65.25000,LMGGN1S50E1000025
0,99.8,80.381053,64.1,10.212795,269.0,45.080,5.687643,0.0,7.768781,11.250000,16.212281,18.0,15.0,284.0,356.0,330.179577,0.0,59.950859,269.0,199.0,0.0,2.0,285.0,0.09,0.011719,0.00,0.011392,NaN,0,NaN,3.773040,120.201053,21.0,0,21.0,0.747222,NaN,3.803394,285.0,6.0,5.723860,3.7,5.820004e-01,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,26.834779,255,262.0,27.95625,8.802481e-02,-35.38125,8.078139,60.46875,LMGGN1S51E1000017
0,62.4,49.932050,35.8,5.966151,1263.0,81.144,6.303968,0.0,11.427337,54.756944,21.430872,26.0,15.0,1273.0,358.0,342.680283,0.0,23.993986,1263.0,776.0,0.0,2.0,1273.0,0.08,0.012184,0.00,0.007976,NaN,0,NaN,9.724544,524.998273,86.0,0,86.0,3.508333,NaN,18.760547,1274.0,9.8,6.104631,4.1,1.068143e+00,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,26.203253,1168,1259.0,45.39375,1.975298e-17,-58.21875,10.619107,93.88125,LMGGN1S52E1000026
0,58.2,58.088889,58.0,0.078174,6.0,0.000,0.000000,0.0,0.000000,0.088889,7.000000,8.0,6.0,9.0,347.0,269.666667,0.0,152.887213,6.0,4.0,0.0,2.0,9.0,0.02,0.011111,0.01,0.003333,NaN,0,NaN,0.000000,0.000000,0.0,0,0.0,0.016667,NaN,0.030838,9.0,6.4,6.400000,6.4,0.000000e+00,0.0,0.0,Na

In [7]:
hiveContext = HiveContext(sc)

In [9]:
sc.stop()

In [ ]:
res_daily.drop(labels='time_label', axis=1, inplace=True)
# spark_df = hiveContext.createDataFrame(res_daily)

In [ ]:
spark_df.registerTempTable('update_dataframe')
query = hiveContext.sql("""SELECT COUNT(*) FROM tsp_tbls.daily_stats""")
query.show()

hiveContext.sql("""INSERT OVERWRITE TABLE tsp_tbls.daily_stats
                   PARTITION (time_label='2015-11-18')
                   SELECT * FROM update_dataframe""")

query = hiveContext.sql(""" SELECT COUNT(*) FROM tsp_tbls.daily_stats""")
query.show()

In [72]:
sc.stop()

### backup functions

In [4]:
ret, out, err = run_cmd(['hdfs', 'dfs', '-ls', '/data/ag/by-day/ag_201801*.csv'])
lines = out.split('\n')
files = sorted(['hdfs://namenode:8020' + line.split()[-1] for line in lines[:-1]])
files[0]

'hdfs://namenode:8020/data/ag/by-day/ag_20180101.csv'

In [2]:
import subprocess
def run_cmd(args_list):
#     print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s_output, s_err = proc.communicate()
    s_return =  proc.returncode
    return s_return, s_output, s_err 

# (ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', '/data/ag/by-day/'])
# lines = out.split('\n')